In [1]:
import numpy as np
import pandas as pd
import base64
import hashlib
import hmac
import json
import time
import httplib2

In [2]:
#wage_calculate_with_adu의 부분입니다.

work_time_sim = pd.read_pickle("work_time_sim.pkl")

week_all=[]
week_num = ["1st","2nd","3rd","4th"]
day_of_week = [0,1,2,3,4,5,6]
for i in week_num :
    for j in day_of_week :
        week_all.append(int(list(work_time_sim[i])[j]))

week_1st = week_all[0:7]
week_2nd = week_all[7:14]
week_3rd = week_all[14:21]
week_4th = week_all[21:28]

work_time = pd.DataFrame(week_1st, columns = ['1st'], index = ['mon','tue', 'wen' , 'thu' , 'fri' , 'sat' , 'sun'])
work_time['2nd'] = week_2nd
work_time['3rd'] = week_3rd
work_time['4th'] = week_4th
work_time['H'] = (9500)

In [3]:
#wage_calculate_with_adu의 부분입니다.

a = work_time['1st']
b = work_time['2nd']
c = work_time['3rd']
d = work_time['4th']
per_hour = work_time['H'][1]

week1 = a.sum()
week2 = b.sum()
week3 = c.sum()
week4 = d.sum()

x = [week1,week2,week3,week4]
work_time = []
work_wage = []

for i in x :
    if i <= 15 :
        i = i
        work_time.append(i)
        work_wage.append(i*per_hour)
        
    elif i <= 40 :
        i = i * 1.2
        work_time.append(i)
        work_wage.append(i*per_hour)
        
    else:
        i = i + 40 * 0.2
        work_time.append(i)
        work_wage.append(i*per_hour)
        
    
time_sum = sum(work_time)
work_sum = sum(work_wage)

In [6]:
#coinone_api_krw의 부분입니다

ACCESS_TOKEN = '52b36f87-c585-4786-85da-7f4af2208585'
SECRET_KEY = bytes('d2097823-9f5d-4851-812c-eea7f866012a', 'utf-8')


def get_encoded_payload(payload):
    payload['nonce'] = int(time.time() * 1000)

    dumped_json = json.dumps(payload)
    encoded_json = base64.b64encode(bytes(dumped_json, 'utf-8'))
    return encoded_json


def get_signature(encoded_payload):
    signature = hmac.new(SECRET_KEY, encoded_payload, hashlib.sha512)
    return signature.hexdigest()


def get_response(action, payload):
    url = '{}{}'.format('https://api.coinone.co.kr/', action)

    encoded_payload = get_encoded_payload(payload)

    headers = {
        'Content-type': 'application/json',
        'X-COINONE-PAYLOAD': encoded_payload,
        'X-COINONE-SIGNATURE': get_signature(encoded_payload),
    }

    http = httplib2.Http()
    response, content = http.request(url, 'POST', body=encoded_payload, headers=headers)

    return content


c_value = get_response(action='v2/account/balance', payload={'access_token': ACCESS_TOKEN,})
c_value_str = str(c_value[71:74])
c_value_str2 = c_value_str[2:5]
c_value_ = float(c_value_str2)


3.0

In [8]:
if work_sum < c_value_krw :
    transfer_possibility = "possible"
else :
    transfer_possibility = "impossible"

transfer_possibility

'impossible'